In [55]:
import numpy as np

In [56]:
class DecisionTree:
    def __init__(self):
        """the decision tree is a dictionary, the key is the value 
        of the feature, the value is the subtree"""
        self.tree = {}

    def entropy(self, x):  # x can be a 1d array
        """return H(X)"""
        rv, counts = np.unique(x, return_counts=True)
        pdf = counts / len(x)
        entropy = 0
        for prob in pdf:
            # if the probability is 0, the entropy is 0 according to Lopital's rule
            if prob > 0:
                entropy -= prob * np.log2(prob)
        return entropy

    def info_gain(self, x, y):
        """return H(Y) - H(Y|X)"""
        H_Y = self.entropy(y)
        entropy_current = []
        x_rv, counts = np.unique(x, return_counts=True)
        x_pdf = counts / len(x)
        for value in x_rv:
            H_Y_X = self.entropy(y[x == value])
            entropy_current.append(H_Y_X)
        H_Y_X = np.sum((np.array(entropy_current)) * x_pdf)
        return H_Y - H_Y_X

    def argmax_ig(self, features_data, lables):
        """find the index of the feature that has the largest information gain"""
        ig_max = 0
        feature_index = None
        n_features = len(features_data[0])
        for i in range(n_features):
            ig_current = self.info_gain(features_data[:, i], lables)
            if ig_current > ig_max:
                ig_max = ig_current
                feature_index = i
        return feature_index

    def train(self, features, lables):
        """build a decision tree(the dictionary) recursively and return it"""
        ig_max_id = self.argmax_ig(features, lables)
        if ig_max_id is None: # no information gain  
            # return the most frequent class
            return np.argmax(np.bincount(lables))
        else:# there is still information gain
            tree = {}
            rv_selected_feature = np.unique(features[:, ig_max_id])
            for x_i in rv_selected_feature:
                # find the samples that have the value of the feature
                index = np.where(features[:, ig_max_id] == x_i)
                features_sub = features[index]
                lables_sub = lables[index]
                # delete the feature column
                features_sub = np.delete(features_sub, ig_max_id, axis=1)
                tree[x_i] = self.train(features_sub, lables_sub)
        return tree

    def fit(self, features, lables):
        self.tree = self.train(features, lables)

    def predict(self, tree, sample):
        """predict the class of the sample"""
        if isinstance(tree, dict):
            for key, value in tree.items():
                if key == sample[0]:
                    return self.predict(value, sample[1:])
        else:
            return tree

### Test case:

In [59]:
original_data = np.loadtxt('lenses.data',dtype='str')
tree = DecisionTree()
features = original_data[:,1:-1].astype('int')
feature_names = ['feature1', 'feature2', 'feature3', 'feature4']
lables = original_data[:,-1].astype('int')
tree.fit(features, lables)

In [60]:
def show_tree(tree, depth=0):
    """print the tree"""
    if isinstance(tree, dict):# it is a subtree
        for key, value in tree.items():
            print('\t' * depth + 'if '+ str(feature_names[key]) + ' = ' + str(key)+ ':')
            show_tree(value, depth + 1)
    else:#it is a leaf
        print('\t' * depth + 'class --> ' + str(tree))

show_tree(tree.tree)

if feature2 = 1:
	class --> 3
if feature3 = 2:
	if feature2 = 1:
		if feature2 = 1:
			class --> 2
		if feature3 = 2:
			class --> 2
		if feature4 = 3:
			if feature2 = 1:
				class --> 3
			if feature3 = 2:
				class --> 2
	if feature3 = 2:
		if feature2 = 1:
			class --> 1
		if feature3 = 2:
			if feature2 = 1:
				class --> 1
			if feature3 = 2:
				class --> 3
			if feature4 = 3:
				class --> 3
